## Mexico Police Department Incidents

### Import modules

In [1]:
import folium
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Geolocation

In [2]:
# Mexico coordinates
MEXICO_CORRDINATES = (23.6345, -102.5528)

In [3]:
# Create a Map instance with center at Mexico (location= [23.6345, -102.5528]) and zoom level 5
mexico_map = folium.Map(location=MEXICO_CORRDINATES, zoom_start=5)
mexico_map

In [4]:
# Create a map of Mexico with a zoom level of 6.3
mexico_map = folium.Map(location=MEXICO_CORRDINATES, zoom_start=6.3, tiles='OpenStreetMap')
mexico_map

### Map with Marker

**San Francisco Police Department Incidents 2016 - provided from the San Francisco public data portal. Incidents originate from the San Francisco Police Department's (SFPD) crime incident reporting system. Updated daily, showing data for the entire year 2016. Address and location have been anonymized by moving to the middle of the block (mid-block) or to an intersection (intersection)**

#### Read the data

In [5]:
df_incidents = pd.read_csv('data/Police_Department_Incidents_-_Previous_Year__2016_.csv')

So each row consists of 13 features:
> 1. **IncidntNum**: Incident Number
> 2. **Category**: Category of crime or incident
> 3. **Descript**: Description of the crime or incident
> 4. **DayOfWeek**: The day of week on which the incident occurred
> 5. **Date**: The Date on which the incident occurred
> 6. **Time**: The time of day on which the incident occurred
> 7. **PdDistrict**: The police department district
> 8. **Resolution**: The resolution of the crime in terms whether the perpetrator was arrested or not
> 9. **Address**: The closest address to where the incident took place
> 10. **X**: The longitude value of the crime location 
> 11. **Y**: The latitude value of the crime location
> 12. **Location**: A tuple of the latitude and the longitude values
> 13. **PdId**: The police department ID

In [6]:
df_incidents.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150500 entries, 0 to 150499
Data columns (total 13 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   IncidntNum  150500 non-null  int64  
 1   Category    150500 non-null  object 
 2   Descript    150500 non-null  object 
 3   DayOfWeek   150500 non-null  object 
 4   Date        150500 non-null  object 
 5   Time        150500 non-null  object 
 6   PdDistrict  150499 non-null  object 
 7   Resolution  150500 non-null  object 
 8   Address     150500 non-null  object 
 9   X           150500 non-null  float64
 10  Y           150500 non-null  float64
 11  Location    150500 non-null  object 
 12  PdId        150500 non-null  int64  
dtypes: float64(2), int64(2), object(9)
memory usage: 14.9+ MB


In [7]:
df_incidents.head()

,IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location,PdId
0,120058272,WEAPON LAWS,POSS OF PROHIBITED WEAPON,Friday,01/29/2016 12:00:00 AM,11:00,SOUTHERN,"ARREST, BOOKED",800 Block of BRYANT ST,-122.403405,37.775421,"(37.775420706711, -122.403404791479)",12005827212120
1,120058272,WEAPON LAWS,"FIREARM, LOADED, IN VEHICLE, POSSESSION OR USE",Friday,01/29/2016 12:00:00 AM,11:00,SOUTHERN,"ARREST, BOOKED",800 Block of BRYANT ST,-122.403405,37.775421,"(37.775420706711, -122.403404791479)",12005827212168
2,141059263,WARRANTS,WARRANT ARREST,Monday,04/25/2016 12:00:00 AM,14:59,BAYVIEW,"ARREST, BOOKED",KEITH ST / SHAFTER AV,-122.388856,37.729981,"(37.7299809672996, -122.388856204292)",14105926363010
3,160013662,NON-CRIMINAL,LOST PROPERTY,Tuesday,01/05/2016 12:00:00 AM,23:50,TENDERLOIN,NONE,JONES ST / OFARRELL ST,-122.412971,37.785788,"(37.7857883766888, -122.412970537591)",16001366271000
4,160002740,NON-CRIMINAL,LOST PROPERTY,Friday,01/01/2016 12:00:00 AM,00:30,MISSION,NONE,16TH ST / MISSION ST,-122.419672,37.765050,"(37.7650501214668, -122.419671780296)",16000274071000


In [8]:
df_incidents.shape

(150500, 13)

#### Plot crimes on the map

In [9]:
# There are 150500 crimes in our dataset.
# Get the first 100 crimes in this dataset.
limit = 100
df_incidents = df_incidents.iloc[0:100, :]
df_incidents.shape

(100, 13)

In [10]:
# Create a map of San Francisco with center at San Francisco (location= [37.77, -122.42]) and zoom level 12
SF_COORDINATES = (37.77, -122.42)
sanfran_map = folium.Map(location=SF_COORDINATES, zoom_start=12)
sanfran_map

In [11]:
# Add 100 crimes to the map (with marker)
incidents = folium.map.FeatureGroup()

for lat, lng, in zip(df_incidents.Y, df_incidents.X):
    incidents.add_child(
        folium.features.Marker(
            [lat, lng],
            radius=5,
            color='yellow',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6
        )
    )

sanfran_map.add_child(incidents)

In [12]:
# Add pop-up text to each marker on the map
incidents = folium.map.FeatureGroup()

for lat, lng, in zip(df_incidents.Y, df_incidents.X):
    incidents.add_child(
        folium.features.Marker(
            [lat, lng],
            radius=5, 
            color='yellow',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6
        )
    )

latitudes = list(df_incidents.Y)
longitudes = list(df_incidents.X)
labels = list(df_incidents.Category)

for lat, lng, label in zip(latitudes, longitudes, labels):
    folium.Marker([lat, lng], popup=label).add_to(sanfran_map)    
    
sanfran_map.add_child(incidents)

In [13]:
# To make the map less crowded, let's remove the location markers, and add pop-up text to each circle marker.
sanfran_map = folium.Map(location=SF_COORDINATES, zoom_start=12)

for lat, lng, label in zip(df_incidents.Y, df_incidents.X, df_incidents.Category):
    folium.features.Marker(
        [lat, lng],
        radius=5, 
        color='yellow',
        fill=True,
        popup=label,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(sanfran_map)

sanfran_map

#### How many crimes

In [14]:
# Group the markers into different clusters. Each cluster will be identified by how many crimes it has.
# Hint: Use MarkerCluster

from folium import plugins

sanfran_map = folium.Map(location = SF_COORDINATES, zoom_start = 12)
incidents = plugins.MarkerCluster().add_to(sanfran_map)

for lat, lng, label, in zip(df_incidents.Y, df_incidents.X, df_incidents.Category):
    folium.Marker(
        location=[lat, lng],
        icon=None,
        popup=label,
    ).add_to(incidents)

sanfran_map